## 0. Imports

In [23]:
from fastai.collab import *
from fastai.tabular import *

In [10]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [41]:
from pathlib import Path
import pandas as pd
from utils.generic import *
from utils.nn import *

## 1. Dataset

In [12]:
path = Path('./data')

MovieLens Latest Small

In [15]:
ratings_latest, movies_latest = import_datasets(path/'ml-latest-small')
ratings_latest.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


MovieLens 1M

In [16]:
ratings_1m, movies_1m = import_datasets(path/'ml-1m', ratings='ratings.dat', movies='movies.dat')
ratings_1m.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


MovieLens 100K

In [19]:
ratings_100k, movies_100k = import_datasets(path/'ml-100k', ratings='u.data', movies='u.item')
ratings_100k.head()

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [27]:
data = CollabDataBunch.from_df(ratings_df, seed=42, valid_pct=0.1)

In [29]:
y_range = [0, 5.5]

In [77]:
learn_fastai = collab_learner(data, n_factors=40, y_range=y_range, wd=1e-1)

In [78]:
learn_fastai.model

EmbeddingDotBias(
  (u_weight): Embedding(944, 40)
  (i_weight): Embedding(1669, 40)
  (u_bias): Embedding(944, 1)
  (i_bias): Embedding(1669, 1)
)

In [ ]:
learn.mode

In [ ]:
learn

In [32]:
learnfastai.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.961065,0.936281,00:07
1,0.850279,0.869763,00:07
2,0.770742,0.820022,00:07
3,0.674680,0.800497,00:07
4,0.539882,0.800707,00:07


In [31]:
learn_fastai.model

EmbeddingDotBias(
  (u_weight): Embedding(944, 40)
  (i_weight): Embedding(1669, 40)
  (u_bias): Embedding(944, 1)
  (i_bias): Embedding(1669, 1)
)

In [39]:
ratings_df = ratings_100k # change here to change dataset
ratings_data = CollabData(ratings_df, test_size=0.2, bs=256)
ratings_data.show_batch()

   userId  movieId  rating
0     642      215       4
1     457       78       5
2     467       68       4
3     935      677       3
4     710      960       5
5     753      358       3
6     439      309       3
7     436      811       3
8     895      199       4
9      58      854       4


In [70]:
ratings_data.y_range = [0.5, 5]

## 2. Learner & Model

In [87]:
n_factors = 40
embedding_dropout = 0.02
hidden = [200, 100]
dense_dropouts = [0.35, 0.25]

learn = CollabLearner(ratings_data, EmbedDot, n_factors=n_factors, embedding_dropout=embedding_dropout, hidden=hidden, dense_dropouts=dense_dropouts, normalize_batches=False)
learn.model

EmbedDot(
  (u): Embedding(943, 40)
  (m): Embedding(1682, 40)
  (u_bias): Embedding(943, 1)
  (m_bias): Embedding(1682, 1)
)

In [84]:
list(learn.model.children())

[Embedding(943, 40),
 Embedding(1682, 40),
 Embedding(943, 1),
 Embedding(1682, 1)]

## 4. Train

### 4.1 Static Learning Rate

In [38]:
learn.model.random_weights()

n_epochs = 10
lr = 3e-3
wd = 5e-1

learn.fit(n_epochs, lr, wd)

[0.003, 0.003, 0.003, 0.003, 0.003, 0.003]
epoch: 0, train loss: 1.0317901372909546, validation loss: 0.9498947858810425
epoch: 1, train loss: 0.9097902178764343, validation loss: 0.9060656428337097
epoch: 2, train loss: 0.8605380654335022, validation loss: 0.8812305927276611
epoch: 3, train loss: 0.8426979184150696, validation loss: 0.8755730986595154
epoch: 4, train loss: 0.8329564332962036, validation loss: 0.8802736401557922
epoch: 5, train loss: 0.8287291526794434, validation loss: 0.8774242997169495
epoch: 6, train loss: 0.8227400779724121, validation loss: 0.8783583045005798
epoch: 7, train loss: 0.8182969689369202, validation loss: 0.8822575807571411
epoch: 8, train loss: 0.816341757774353, validation loss: 0.886233389377594
epoch: 9, train loss: 0.8117641806602478, validation loss: 0.8785882592201233


### 4.2 Dynamic Learning Rate with One-Cycle Policy

In [22]:
learn.model.random_weights()

n_epochs = 8
lrs = [3e-3, 3e-3, 1e-3, 3e-3, 5e-3]
# lrs = 5e-3
wd = 5e-1

learn.fit_one_cycle(cycle_len=n_epochs, lr_max=lrs, wd=wd, div_factor=4, final_div=5, pct_start=0.7, moms=(0.92, 0.89))

[0.003, 0.003, 0.001, 0.003, 0.005]
epoch: 0, train loss: 1.1480129957199097, validation loss: 1.1083710193634033
epoch: 1, train loss: 1.0959831476211548, validation loss: 1.074352741241455
epoch: 2, train loss: 1.0402979850769043, validation loss: 1.007962703704834
epoch: 3, train loss: 0.9727092981338501, validation loss: 0.9640842080116272
epoch: 4, train loss: 0.9431629180908203, validation loss: 0.9552000761032104
epoch: 5, train loss: 0.9341853857040405, validation loss: 0.9465305209159851
epoch: 6, train loss: 0.9218698143959045, validation loss: 0.9419063329696655
epoch: 7, train loss: 0.8996491432189941, validation loss: 0.9376044869422913


In [68]:
learn.model.random_weights()

n_epochs = 5
lrs = [5e-3, 5e-3, 3e-3, 3e-3, 4e-3, 5e-3]
wd = 5e-1

learn.fit_one_cycle(cycle_len=n_epochs, lr_max=lrs, wd=wd, div_factor=5, final_div=7, pct_start=0.2, moms=(0.92, 0.88))

[0.005, 0.005, 0.003, 0.003, 0.004, 0.005]
epoch: 0, train loss: 1.0614076852798462, validation loss: 0.9783969521522522
epoch: 1, train loss: 0.9228038787841797, validation loss: 0.8997908234596252
epoch: 2, train loss: 0.8608425259590149, validation loss: 0.8736365437507629
epoch: 3, train loss: 0.8262368440628052, validation loss: 0.8710839748382568
epoch: 4, train loss: 0.7957017421722412, validation loss: 0.8691396713256836


In [52]:
# EmbedDot

In [91]:
learn_fastai.opt

functools.partial(<class 'torch.optim.adam.Adam'>, betas=(0.9, 0.99))

In [88]:
learn.model.random_weights()

n_epochs = 5
lrs = 5e-3
wd = 1e-1

learn.fit_one_cycle(cycle_len=n_epochs, lr_max=lrs, wd=wd, div_factor=5, final_div=7, pct_start=0.2, moms=(0.92, 0.88))

[0.005, 0.005, 0.005, 0.005]


/home/stefano/code/projects_school/collaborative-filtering/.env/lib/python3.6/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/stefano/code/projects_school/collaborative-filtering/.env/lib/python3.6/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/stefano/code/projects_school/collaborative-filtering/.env/lib/python3.6/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([32])) that is different to the

epoch: 0, train loss: 2.5430023670196533, validation loss: 2.0678741931915283
epoch: 1, train loss: 1.6426723003387451, validation loss: 1.408429503440857
epoch: 2, train loss: 1.3760732412338257, validation loss: 1.3455182313919067
epoch: 3, train loss: 1.3498095273971558, validation loss: 1.3359943628311157
epoch: 4, train loss: 1.345117211341858, validation loss: 1.333261489868164


## 5. Predict

In [134]:
learn.get_preds('val')

array([[3.85365057, 5.        ],
       [3.82093   , 4.5       ],
       [3.35521889, 4.        ],
       ...,
       [3.80756617, 4.5       ],
       [3.6175766 , 3.        ],
       [3.52774763, 3.        ]])

## 6. Fine Tuning

In [10]:
# TODO

## 7. Evaluation

In [11]:
# TODO